In [5]:
from models.yolo import Detect
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xyxy2xywhn
import coremltools
from PIL import Image
import torch
import numpy as np
import random
import cv2
import os
import shutil
import PIL.Image

In [20]:
MODE = 'debug'

COREML_MODEL = "/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211116/weights/best.mlmodel"
IMAGE_FOLDER = "/Users/zhenyu/Desktop/val/"
OUT_FOLDER = "/Users/zhenyu/Desktop/test/"

SAVE_IMG = True
VIEW_IMG = False
SAVE_TXT = True
CAT_NAMES = ['Screw', 'unknown']
COLORS = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(CAT_NAMES))]
PATH = "./"
# ANCHORS = ([116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]) # from <model>.yml
ANCHORS = ([2.375,3.375, 5.5,5, 4.75,11.75], 
           [6,4.25, 5.375,9.5, 11.25,8.5625], 
           [4.375,9.40625, 9.46875,8.25, 7.43750,16.93750],
           [6.81250,9.60938, 11.54688,5.93750, 14.45312,12.37500])
IMG_SIZE = (2304, 3072)
nc = len(CAT_NAMES)
nl = len(ANCHORS)
na = len(ANCHORS[0]) // 2
no = nc + 5  # number of outputs per anchor
grid = [torch.zeros(1)] * nl  # init grid
anchor_grid = [torch.zeros(1)] * nl
a = torch.tensor(ANCHORS).float().view(nl, -1, 2)
anchor_grid = a.clone().view(nl, 1, -1, 1, 1, 2)
stride = [8, 16, 32, 64] # check your model config
conf_thres = .2

In [21]:
def load_image(path, resize_to=None):
    # resize_to: (Width, Height)
    img = PIL.Image.open(path)
    if resize_to is not None:
        img = img.resize(resize_to, PIL.Image.ANTIALIAS)
    img_np = np.array(img).astype(np.float32)
    return img_np, img

def make_grid(nx=20, ny=20, i=0):
    yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)])
    grid = torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()
    anchor_grid = (a[i].clone() * stride[i]).view((1, na, 1, 1, 2)).expand(1, na, ny, nx, 2).float()
    print(anchor_grid.shape)
    return grid, anchor_grid


def resize_image(source_image):
    # background = source_image.crop((0,0, 2560, 2560))
    background = PIL.Image.new('RGB', IMG_SIZE, "black")
    source_image.thumbnail(IMG_SIZE)
    (w, h) = source_image.size
    background.paste(source_image, (int((IMG_SIZE[0] - w) / 2), int((IMG_SIZE[1] - h) / 2 )))
    return background

In [22]:
def eval(file_name):   
    source = PIL.Image.open(os.path.join(IMAGE_FOLDER, file_name))
    resized = resize_image(source.copy())

    predictions = model.predict({'image': resized})

    z = []  # inference output
    x = []
    for head in ['var_1763', 'var_1778', 'var_1793', 'var_1808']:
        x.append(torch.Tensor(predictions[head]))
    # x.reverse()

    for i in range(nl):
        bs, _, ny, nx, _ = x[i].shape

        if grid[i].shape[2:4] != x[i].shape[2:4]:
            print(grid[i].shape)
            print(anchor_grid[i].shape)
            grid[i], anchor_grid = make_grid(nx, ny, i)

        y = x[i].sigmoid()
        y[..., 0:2] = (y[..., 0:2] * 2. - 0.5 + grid[i]) * stride[i]  # xy
        y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * anchor_grid  # wh
        z.append(y.view(bs, -1, no))
#         break
    
    pred = torch.cat(z, 1)

    pred = non_max_suppression(pred, 0.2, 0.3, classes=None, agnostic=False)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        p, s = "./", ""
        label=[]

        if det is not None and len(det):
            # Rescale boxes from img_size to im0 size
#             det = det[((det[:, 0]-det[:, 2])*(det[:, 1]-det[:, 3])) > 80]
            for *xyxy, conf, cls in det:
                if SAVE_TXT:
                    xywh = xyxy2xywhn(torch.tensor(xyxy).view(1, 4), w=IMG_SIZE[0], h=IMG_SIZE[1]).view(-1).tolist()
                    label.append(('%g ' * 5 + '\n') % (cls, *xywh))
                if SAVE_IMG:
                    draw = PIL.ImageDraw.Draw(source)
                    draw.rectangle(np.array(torch.tensor(xyxy).view(2,2)), outline='red')
        if SAVE_TXT:
            with open(os.path.join(OUT_FOLDER, '{}.txt'.format(file_name[:-4])), 'a') as f:
                for line in label:
                    f.write(line)
        if SAVE_IMG:
            source.save(os.path.join(OUT_FOLDER, '{}.jpg'.format(file_name[:-4])))
    return predictions, z, x, pred

In [ ]:
def eval_test(file_name):   
    source = PIL.Image.open(os.path.join(IMAGE_FOLDER, file_name))
    resized = resize_image(source)

    predictions = model.predict({'image': resized})
    box = xywh2xyxy(y['coordinates'] * [[w, h, w, h]])  # xyxy pixels
    conf, cls = y['confidence'].max(1), y['confidence'].argmax(1).astype(np.float)
    y = np.concatenate((box, conf.reshape(-1, 1), cls.reshape(-1, 1)), 1)

In [6]:
model = coremltools.models.model.MLModel(COREML_MODEL)

In [23]:
predictions, z, x, pred = eval('1_4032.jpg')

torch.Size([1])


UnboundLocalError: local variable 'anchor_grid' referenced before assignment

In [15]:
anchor_grid = anchor_test

In [17]:
anchor_grid.shape

torch.Size([1, 3, 384, 288, 2])

In [13]:
anchor_test = (a[0].clone() * stride[0]).view(1, na, 1, 1, 2).expand((1, na, 384, 288, 2)).float()

In [50]:
for each in predictions.keys():
    print('{}:{}'.format(each, predictions[each].shape))

var_1808:(1, 3, 48, 36, 7)
var_1778:(1, 3, 192, 144, 7)
var_1763:(1, 3, 384, 288, 7)
var_1793:(1, 3, 96, 72, 7)


In [26]:
from models.experimental import attempt_load  # scoped to avoid circular import
w = '/Users/zhenyu/Box/MLProject:IphoneAOI/weights/yolov5l6_3072*2304_20211116/weights/best.pt'
model_pt = torch.jit.load(w) if 'torchscript' in w else attempt_load(w)

Fusing layers... 
Model Summary: 476 layers, 76126356 parameters, 0 gradients, 110.0 GFLOPs


In [27]:
model_pt.model[-1].anchors

tensor([[[ 2.37500,  3.37500],
         [ 5.50000,  5.00000],
         [ 4.75000, 11.75000]],

        [[ 6.00000,  4.25000],
         [ 5.37500,  9.50000],
         [11.25000,  8.56250]],

        [[ 4.37500,  9.40625],
         [ 9.46875,  8.25000],
         [ 7.43750, 16.93750]],

        [[ 6.81250,  9.60938],
         [11.54688,  5.93750],
         [14.45312, 12.37500]]])

In [30]:
model

input {
  name: "image"
  type {
    imageType {
      width: 2304
      height: 3072
      colorSpace: RGB
    }
  }
}
output {
  name: "var_1763"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_1778"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_1793"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
output {
  name: "var_1808"
  type {
    multiArrayType {
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==1.10.0+cu102"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "5.1.0"
  }
}

In [25]:
print(model.model[-1].grid[0].shape)
print(model.model[-1].grid[1].shape)
print(model.model[-1].grid[2].shape)
print(model.model[-1].grid[3].shape)

torch.Size([1, 3, 392, 296, 2])
torch.Size([1, 3, 196, 148, 2])
torch.Size([1, 3, 98, 74, 2])
torch.Size([1, 3, 49, 37, 2])


In [26]:
ANCHORS = ([2.375,3.375, 5.5,5, 4.75,11.75], 
           [6,4.25, 5.375,9.5, 11.25,8.5625], 
           [4.375,9.40625, 9.46875,8.25, 7.43750,16.93750],
           [6.81250,9.60938, 11.54688,5.93750, 14.45312,12.37500])
a = torch.tensor(ANCHORS).float().view(4, -1, 2)

In [28]:
stride = [8, 16, 32, 64]

In [33]:
a[0]*torch.tensor(stride)[0]

tensor([[19., 27.],
        [44., 40.],
        [38., 94.]])

In [31]:
a.shape

torch.Size([4, 3, 2])

In [32]:
torch.tensor(stride).shape

torch.Size([4])

In [36]:
len(a[0])//2

1